# 15章 拡張性

レストランで各チェーン店に通知を行う機能を実装したとする

In [ ]:
import datetime

def declear_special(dish: str, start_datetime: datetime.datetime, end_datetime: datetime.datetime):
    # 特別メニューが追加されたことを各チェーン店に通知
    pass

def delete_menu(dish: str):
    # 材料の在庫がないため、メニューから削除することを通知
    pass

def check_for_expired_ingredients():
    # 24時間ごとに呼び出されて材料の賞味期限が切れることを通知
    pass

たとえば、declear_specialをメールで通知したい機能を追加する場合以下のようになる

In [ ]:
def declear_special(dish: str, 
                    start_datetime: datetime.datetime,
                    end_datetime: datetime.datetime,
                    email: str):
    # 新しいメニューが通知されたことを各チェーン店に通知
    pass

さらに以下のような機能追加を要求されたことを考える
- 営業チームにも在庫切れと賞味期限切れを通知する
- レストランの顧客に特別メニューを通知する
- チェーン店ごとに異なるAPIをサポートする
- 新メニューをマーケターと店長のみに通知する

上記の機能を追加するとdeclear_special, delete_menu, check_for_expired_ingredientsの関数が肥大化してしまい  
また通知するという機能が各関数ごとに独自で持っているため、コード修正の際にミスが発生する可能性が高くなる  

In [ ]:
# declaer_specialに目を当てつと以下のような関数に肥大化していまう
def declear_special(dish: str, 
                    start_datetime: datetime.datetime,
                    end_datetime: datetime.datetime,
                    email: str,
                    send_customer: bool,
                    phone_number: str):
    # 新しいメニューが通知されたことを各チェーン店に通知
    pass

通知に関する内容は大きく3つに分類できると考えた場合  
- 通知の内容 -> メニューの追加、メニューの削除など
- 通知の手段 -> Email, ショートメッセージなど
- 通知の宛先 -> 店長のメールアドレス, マーケターの電話番号

In [1]:
# それぞれの機能を分解してクラスを設計し直す
from dataclasses import dataclass
from typing import Any, List, Dict, Set, Union

notifications: list[Any] = []

Dish = str
Ingredient = str

## 通知内容のclass

In [ ]:
@dataclass
class NewSpecial:
    dish: Dish
    start_date: datetime.datetime
    end_date: datetime.datetime

@dataclass
class IngredientsOutOfStock:
    ingredients: set[Ingredient]

@dataclass
class IngredientsExpired:
    ingredients: set[Ingredient]

@dataclass
class NewMenuItem:
    dish: Dish

Notification = Union[NewSpecial, IngredientsOutOfStock, IngredientsExpired, NewMenuItem]

## 通知手段のclass

In [ ]:
@dataclass
class Text:
    phone_number: str

@dataclass
class Email:
    email_address: str

@dataclass
class SupplierAPI:
    pass

NotificationMethod = Union[Text, Email, SupplierAPI]

In [ ]:
# 通知方法ごとに関数が異なるのは必然的な複雑性であるため、しかたない
# メッセージ、必要な内容、フォーマットが異なるためである

def notify(notification_method: NotificationMethod, notification: Notification):
    """ショートメッセージとかによる通知
    """
    if isinstance(notification_method, Text):
        send_text(notification_method, notification)
    elif isinstance(notification_method, Email):
        send_email(notification_method, notification)
    elif isinstance(notification_method, SupplierAPI):
        send_to_supplier(notification)
    else:
        raise ValueError("Unsupported Notification Method")

def send_text(text: Text, notification: Notification):
    if isinstance(notification, NewSpecial):
        # ... snip send text ...
        pass
    elif isinstance(notification, IngredientsOutOfStock):
        # ... snip send text ...
        pass
    elif isinstance(notification, IngredientsExpired):
        # ... snip send text ...
        pass
    elif isinstance(notification, NewMenuItem):
        # .. snip send text ...
        pass
    raise NotImplementedError("Notification method not supported")

def send_email(email: Email, notification: Notification):
    # .. similar to send_text ...
    global notifications
    if isinstance(notification, IngredientsExpired):
        # ... snip send text ...
        notifications.append((email.email_address, notification.ingredients))
    if isinstance(notification, NewMenuItem):
        # ... snip send text ...
        notifications.append((email.e

def send_to_supplier(notification: Notification):
    # .. similar to send_text
    global notifications
    if isinstance(notification, IngredientsExpired):
        # ... snip send text ...
        notifications.append(("supplier", notification.ingredients))mail_address, notification.dish))

## 通知の宛先
通知内容ごとに通知のリストを得られるようにしておけばよい

In [ ]:
users_to_notify: dict[type, list[NotificationMethod]] = {
    NewSpecial: [SupplierAPI(), Email("boss@company.org"), Email("marketing@company.org"), Text("555-2345")],
    IngredientsOutOfStock: [SupplierAPI(), Email("boss@company.org")],
    IngredientsExpired: [SupplierAPI(), Email("boss@company.org")],
    NewMenuItem: [Email("boss@company.org"), Email("marketing@company.org")]
}

以上の内容をまとめて通知用の関数を定義することが出来れば終了である

In [ ]:
def send_notification(notification: Notification):
    try:
        users = users_to_notify[type(notification)]
    except KeyError:
        raise ValueError("Unsupported Notification Method")
    for user in users:
        notify(user, notification)

こうすることで他のコードは、通知システムとやり取りするときにsend_notification関数の存在だけを知っておけばよい(関数を1つにまとめることができた)  
また、通知方法や通知内容が追加された場合は個別のclassに追加するだけでよいため拡張性もある  
既存コードへの変更を最小限にながらもコードベースに新しい機能を追加できることを**解放閉鎖の原則**と呼ぶ